# Escrapeo precios mercadona:

https://tienda.mercadona.es/

In [1]:
# Importamos librerías necesarias para escrapear

from selenium import webdriver 

import time  

import requests as req

from bs4 import BeautifulSoup as bs

import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
PATH=ChromeDriverManager().install()



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\HP\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [17]:
driver=webdriver.Chrome(PATH)

driver.get('https://tienda.mercadona.es/')

# Aceptamos las cookies
driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/button[2]').click()
time.sleep(2)

# Introducimos el código postal
driver.find_element_by_xpath('//*[@id="root"]/div[4]/div/div[2]/div/form/div/input').send_keys('28001')

# Continuamos
driver.find_element_by_xpath('//*[@id="root"]/div[4]/div/div[2]/div/form/button').click()
time.sleep(2)

# Vamos a categorías
driver.find_element_by_xpath('//*[@id="root"]/header/div[1]/nav/a[1]').click()
time.sleep(2)

## Aquí comienza el escrapeo
html = [] # Hay 26 categorías y 155 (len(html) = 155)

for i in range(2, 28):
    while 1:
        
        # Vamos sacando el html de cada página
        html.append(bs(driver.page_source, 'html.parser'))
        try:
            # Clicamos al botón de ver siguiente subcategoría
            driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div[1]/div/div/button').click()
            time.sleep(2)
        except:
            break

    # Clicamos en la categoría siguiente
    try:
        driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[1]/ul/li[{}]/div/button/span/label'.format(i)).click()
        time.sleep(2)
    except:
        break
        
driver.quit() # Apagamos el driver

C:\Users\HP\AppData\Local\Temp/ipykernel_6096/3843881553.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)
C:\Users\HP\AppData\Local\Temp/ipykernel_6096/3843881553.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/button[2]').click()
C:\Users\HP\AppData\Local\Temp/ipykernel_6096/3843881553.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="root"]/div[4]/div/div[2]/div/form/div/input').send_keys('28001')
C:\Users\HP\AppData\Local\Temp/ipykernel_6096/3843881553.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="root"]/div[4]/div/div[2]/div/form/button').click()
C:\Users\HP\AppData\Local\Temp/ipykernel_6096/3843881

In [18]:
len(html)

155

In [107]:
# Sacamos el nombre de los productos de todas las subcategorías
nombres = []

for i in range(155):
    nombres.append(html[i].find_all(class_='subhead1-r product-cell__description-name'))

# Desglosamos la lista en una sola
nombres_list = []

for item in nombres:
    nombres_list += item
    
# Sacamos el texto de la lista
nombres = [nombre.text for nombre in nombres_list]

In [108]:
len(nombres)

6589

In [63]:
# Sacamos los precios de todas las subcategorías
precios = []

for i in range(155):
    precios.append(html[i].find_all(class_='product-price__unit-price subhead1-b'))

# Desglosamos la lista en una sola
precios_list = []

for item in precios:
    precios_list += item
    
# Sacamos el texto de la lista
precios = [precio.text for precio in precios_list]

In [64]:
len(precios)

6589

In [81]:
# Sacamos la descripcion de todas las subcategorías
descripcion = []

# Sacamos la descripción de los productos por subcategorías
for i in range(155):
    descripcion.append(html[i].find_all('div', class_='product-format product-format__size--cell'))
    
# Desglosamos la lista en una sola
descripcion_list = []

for item in descripcion:
    descripcion_list += item
    
# Sacamos el texto de la lista
descripcion = [descripcion.text for descripcion in descripcion_list]

In [83]:
len(descripcion)

6589

In [104]:
# Sacamos las URLs de las imágenes de todas las subcategorías
imagenes = []

# Sacamos la URL de los productos por subcategoría
for i in range(155):
    imagenes.append(html[i].find_all(class_='product-cell__image-wrapper'))

# Desglosamos la lista en una sola
imagenes_list = []

for item in imagenes:
    imagenes_list += item
    
# Sacamos la URl de la imagen de cada producto
imagenes = [imagen.find('img').attrs['src'] for imagen in imagenes_list]

In [109]:
len(imagenes)

6589

In [200]:
# Creamos un diccionario con cada uno de los datos del producto

productos = pd.DataFrame({'nombre': nombres,
           'descripcion': descripcion,
           'precio': precios,
           'imagen': imagenes})

In [201]:
# Convertimos los precios en float y los strings los pasamos a minúsculas
productos['precio'] = productos.precio.apply(lambda x: x.replace('€', '').strip())
productos['precio'] = productos.precio.apply(lambda x: float(x.replace(',', '.')))
productos['nombre'] = productos.nombre.apply(lambda x: x.lower())
productos['descripcion'] = productos.descripcion.apply(lambda x: x.lower())

In [202]:
productos.head()

,nombre,descripcion,precio,imagen
0,"aceite de oliva 0,4º hacendado",garrafa 5 l,15.50,https://prod-mercadona.imgix.net/images/af5f40...
1,"aceite de oliva 0,4º hacendado",botella 1 l,3.20,https://prod-mercadona.imgix.net/images/2c743e...
2,aceite de oliva virgen extra hacendado,garrafa 3 l,11.00,https://prod-mercadona.imgix.net/images/84dff1...
3,aceite de oliva virgen extra hacendado,botella 1 l,3.95,https://prod-mercadona.imgix.net/images/95e2ef...
4,aceite de oliva virgen extra hacendado gran se...,botella 750 ml,4.10,https://prod-mercadona.imgix.net/images/c3e884...


In [203]:
# Sacamos las subcategorías
subcategorias = []

for i in range(155):
    subcategorias.append(html[i].find_all(class_='category-detail__title title1-b'))
    
# Desglosamos la lista en una sola
subcategorias_list = []

for item in subcategorias:
    subcategorias_list += item
    
# Sacamos los nombres de lsa subcategorias en minúscula
subcategorias = [subcategoria.text.lower() for subcategoria in subcategorias_list]

In [204]:
len(subcategorias)

155

In [205]:
# Vemos el número de productos por subcategoría
numero_productos = []

for i in range(155):
    numero_productos.append(len(html[i].find_all(class_='product-cell')))

In [206]:
'''Una vez tenemos el número de productos por subcategorías lo que hacemos es multiplicar el nombre de cada subcategoría 
por el número de productos que tiene cada subcategoría, sacamos la lista para implementar una nueva columna en nuestro
DataFrame, lo hacemos con un bucle simultáneo donde i es el índice y j el valor de la variable (ambos iguales)'''
numbers = list(range(155))
subcategoria_columna = []

for i, j in enumerate(numbers):
    subcategoria_columna.append(numero_productos[i] * [subcategorias[j]])
    
# Desglosamos la lista en una sola
subcategorias_columna_list = []

for item in subcategoria_columna:
    subcategorias_columna_list += item

In [207]:
productos['subcategoria'] = subcategorias_columna_list

In [208]:
productos.head()

,nombre,descripcion,precio,imagen,subcategoria
0,"aceite de oliva 0,4º hacendado",garrafa 5 l,15.50,https://prod-mercadona.imgix.net/images/af5f40...,"aceite, vinagre y sal"
1,"aceite de oliva 0,4º hacendado",botella 1 l,3.20,https://prod-mercadona.imgix.net/images/2c743e...,"aceite, vinagre y sal"
2,aceite de oliva virgen extra hacendado,garrafa 3 l,11.00,https://prod-mercadona.imgix.net/images/84dff1...,"aceite, vinagre y sal"
3,aceite de oliva virgen extra hacendado,botella 1 l,3.95,https://prod-mercadona.imgix.net/images/95e2ef...,"aceite, vinagre y sal"
4,aceite de oliva virgen extra hacendado gran se...,botella 750 ml,4.10,https://prod-mercadona.imgix.net/images/c3e884...,"aceite, vinagre y sal"


In [209]:
productos.to_csv(r'C:\Users\HP\Dropbox\Mi PC (DESKTOP-KE4BQ28)\Desktop\Proyecto página web comparador de precios\productos_mercadona.csv', index = False)
productos.to_excel(r'C:\Users\HP\Dropbox\Mi PC (DESKTOP-KE4BQ28)\Desktop\Proyecto página web comparador de precios\productos_mercadona.xlsx', index = False)